# Apply k-NN and SVM for digit recognition

Before applying Deep Learning, first apply the k-NN and SVM algorithms as baseline models.

In [1]:
import os
from mnist import MNIST
import numpy as np
from sklearn import svm
import pandas as pd

In [2]:
mnidata = MNIST(os.getcwd()+'/Data')
train_images, train_labels = mnidata.load_training()
test_images, test_labels = mnidata.load_testing()

In [10]:
# dataset is too large for personal computer. Reduce dataset by using pandas sample function
train_images_sample = pd.DataFrame(train_images).sample(n=1000)
train_labels_sample = pd.Series(train_labels).iloc[train_images_sample.index]
test_images_sample = pd.DataFrame(test_images).sample(n=200)
test_labels_sample = pd.Series(test_labels).iloc[test_images_sample.index]

In [12]:
train_images_sample = train_images_sample.values.tolist()
train_labels_sample = train_labels_sample.values.tolist()
test_images_sample = test_images_sample.values.tolist()
test_labels_sample = test_labels_sample.values.tolist()

k-NN

In [1]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
scores = []
for n in range(1, 30, 2):
    kNN_model = KNeighborsClassifier(n)
    kNN_model.fit(train_images_sample, train_labels_sample)
    score = kNN_model.score(test_images_sample, test_labels_sample)
    print (str(n) + ' nearest neighbors score is :' + str(score))
    scores.append([n,score])
    

SVM prediciton score 0.07


int